# Interactive Visualization Assignment (Plotly + IPyWidgets)

In [1]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MaxAbsScaler

The following makes your colab session act more like a jupyter notebook for displaying plotly and ipywidgets objects correctly. If you are using jupyter, comment it out. (It needs to be called at the top of every cell that outputs a plot that uses widgets.)

```
configure_plotly_browser_state()
```

From [this StackOverflow answer](https://stackoverflow.com/a/47230966).

In [3]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
  from plotly.offline import init_notebook_mode
  init_notebook_mode(connected=False)

### Import the housing.csv data set.

In [4]:
data = pd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/housing.csv")
data.shape


(1460, 81)

### Separate out all the numeric fields into their own data set.

Remove the Id field as it should be categorical and the GarageYrBlt and LotFrontage fields that contains many nulls. Drop any remaining nulls, but only for fields that are numeric. Hint: use subset. Then create a new dataframe with only the numeric fields in it.

In [5]:
numeric = data.select_dtypes(include='number')
numeric = numeric.drop(['Id', 'GarageYrBlt', 'LotFrontage'], axis=1)
numeric.dropna(inplace=True)

### Normalize the numeric data using Scikit-Learn's MaxAbsScaler.

In [6]:
scale = MaxAbsScaler()
X_scale = pd.DataFrame(scale.fit_transform(numeric), columns=numeric.columns)

### Iteratively K-Means cluster the normalized data and generate an interactive line chart showing the average silhouette score for each number of clusters (2 through 20).

In [7]:
n_clusters = range(2,21)
scores = []
for i in n_clusters:
    kmeans = KMeans(n_clusters=i)
    clusters = kmeans.fit_predict(X_scale)
    score = silhouette_score(X_scale, clusters)
    scores.append(score)

In [8]:
scores_list = pd.DataFrame(columns=['Clusters', 'Silhouette score'])
scores_list['Clusters'] = n_clusters
scores_list['Silhouette score'] = scores

In [9]:
fig = px.line(
    scores_list, 
    x='Clusters', 
    y='Silhouette score', 
    title='Avg. Silhouette score per k Cluster',
    template='none'
)
fig.show()

### Choose a number of clusters, run KMeans with that value for k on the scaled data, and add a column to the original housing data set containing the cluster that each record is assigned to.

In [10]:
kmeans = KMeans(n_clusters=4)


numeric['Cluster'] = kmeans.fit_predict(X_scale)
numeric.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,Cluster
0,60,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2,548,0,61,0,0,0,0,0,2,2008,208500,0
1,20,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,2,460,298,0,0,0,0,0,0,5,2007,181500,2
2,60,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2,608,0,42,0,0,0,0,0,9,2008,223500,0
3,70,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,3,642,0,35,272,0,0,0,0,2,2006,140000,2
4,60,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,3,836,192,84,0,0,0,0,0,12,2008,250000,0


### Create an interactive bar chart that shows the average SalePrice of a property by cluster.

You will need to aggregate the data by cluster and average the sale prices before generating your visualization.

In [11]:

cluster_means = numeric.groupby(by='Cluster').mean().reset_index()

In [12]:

fig = px.bar(cluster_means, 
            x='Cluster',
            y='SalePrice',
            title='Avg. Sale Price by Cluster',
            template='none')

fig.update_traces(marker_color='lightskyblue',
                  marker_line_color='black',
                  marker_line_width=1)

fig.show()

### Create another bar chart where the bars are broken down and color-coded by the year the property was sold.

You will need to convert the YrSold field to be categorical in order to separate the bars based on that field.

In [13]:
numeric['YrSold'] = numeric.YrSold.astype(str)
cluster_sold_means = numeric.groupby(['Cluster', 'YrSold']).mean().reset_index()

In [14]:
fig = px.bar(cluster_sold_means,
            x='Cluster',
            y='SalePrice',
            color='YrSold',
            template='none',
            title='Avg. Sale Price by Cluster and Year Sold')

fig.update_layout(barmode='group')
fig.update_traces(marker_line_color='black', marker_line_width=1)

fig.show()

### Add a drop-down widget to the multi-bar chart you created above that lets you choose between 4 numeric fields to represent on the Y axis. 

In [15]:
@interact(Metric=['LotArea', 'OverallQual', 'OverallCond', 'SalePrice'])

def barchart(Metric):
    fig = px.bar(cluster_sold_means,
            x='Cluster',
            y=Metric,
            color='YrSold',
            template='none',
            title='Avg. ' + Metric + ' by Cluster and Year Sold')

    fig.update_layout(barmode='group')
    fig.update_traces(marker_line_color='black', marker_line_width=1)

    fig.show()

interactive(children=(Dropdown(description='Metric', options=('LotArea', 'OverallQual', 'OverallCond', 'SalePr…

### Create a scatter plot that shows the relationship between SalesPrice and LotArea, color-coded by cluster. Add a slider that filters the data by the year the property was sold.

In [16]:

numeric.YrSold = numeric.YrSold.astype(int)
numeric.Cluster = numeric.Cluster.astype(str)

@interact(YrSold = (numeric.YrSold.min(), numeric.YrSold.max()))

def scatterplot(YrSold):
    yr_filtered = numeric[numeric['YrSold'] == YrSold]
    fig = px.scatter(
        yr_filtered,
        x='SalePrice',
        y='LotArea',
        color="Cluster",
        template="none",
        title="Scatter Plot by Cluster",
    )

    fig.update_traces(marker_line_color="black", marker_line_width=1)

    fig.show()

interactive(children=(IntSlider(value=2008, description='YrSold', max=2010, min=2006), Output()), _dom_classes…